In [1]:
# XO's notes on 02/02/2022: This notebook is used to test the effect of parameter selection, 
# in particular, min_samples and min_cluster_size, on the
# results of HDBSCAN on velocities/actions for
# a typical sample of Gaia EDR3 astomety + DR2 radial velocities
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Ellipse
from matplotlib.colors import ListedColormap, Normalize, LogNorm
from scipy.interpolate import interpn
import seaborn as sns

cmap = ListedColormap(sns.color_palette("colorblind",256))

label_size = 24
matplotlib.rc('font', size=label_size) 

import sklearn
# print(sklearn.__version__)
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import hdbscan

import warnings
warnings.filterwarnings("once")

In [2]:
# Import data
cm_vel_all = pd.read_hdf('../data/dr3_near_vel_plxzp_g2_only.h5')
orb_param_all = pd.read_hdf('../data/dr3_orb_param_err_g2_only.h5')

/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
# Set all -9999 values in APOGEE and RAVE6 cnn [Fe/H] and alpha to np.nan for easier calculation later
ind_ap_999 = np.where(cm_vel_all.loc[:,'m_h_ap'] < -100)[0]
ind_r6c_999 = np.where(cm_vel_all.loc[:,'m_h_r6c'] < -100)[0]
ind_ap17_999 = np.where(cm_vel_all.loc[:,'m_h_ap17'] < -100)[0]
cm_vel_all.loc[ind_ap_999,'m_h_ap'] = np.nan
cm_vel_all.loc[ind_r6c_999,'m_h_r6c'] = np.nan
cm_vel_all.loc[ind_ap17_999,'m_h_ap17'] = np.nan

ind_ap_999 = np.where(cm_vel_all.loc[:,'m_h_err_ap'] < -100)[0]
ind_ap17_999 = np.where(cm_vel_all.loc[:,'m_h_err_ap17'] < -100)[0]
ind_r6c_999 = np.where(cm_vel_all.loc[:,'m_h_err_r6c'] < -100)[0]
ind_l6s_999 = np.where(cm_vel_all.loc[:,'m_h_err_l6s'] < -100)[0]
cm_vel_all.loc[ind_ap_999,'m_h_err_ap'] = np.nan
cm_vel_all.loc[ind_ap17_999,'m_h_err_ap17'] = np.nan
cm_vel_all.loc[ind_r6c_999,'m_h_err_r6c'] = np.nan
cm_vel_all.loc[ind_l6s_999,'m_h_err_l6s'] = np.nan

ind_ap_999 = np.where(cm_vel_all.loc[:,'alpha_m_ap'] < -100)[0]
ind_ap17_999 = np.where(cm_vel_all.loc[:,'alpha_m_ap17'] < -100)[0]
ind_r6c_999 = np.where(cm_vel_all.loc[:,'alpha_m_r6c'] < -100)[0]
cm_vel_all.loc[ind_ap_999,'alpha_m_ap'] = np.nan
cm_vel_all.loc[ind_ap17_999,'alpha_m_ap17'] = np.nan
cm_vel_all.loc[ind_r6c_999,'alpha_m_r6c'] = np.nan

ind_ap_999 = np.where(cm_vel_all.loc[:,'alpha_m_err_ap'] < -100)[0]
ind_ap17_999 = np.where(cm_vel_all.loc[:,'alpha_m_err_ap17'] < -100)[0]
ind_r6c_999 = np.where(cm_vel_all.loc[:,'alpha_m_err_r6c'] < -100)[0]
ind_l6s_999 = np.where(cm_vel_all.loc[:,'alpha_m_err_l6s'] < -100)[0]
cm_vel_all.loc[ind_ap_999,'alpha_m_err_ap'] = np.nan
cm_vel_all.loc[ind_ap17_999,'alpha_m_err_ap17'] = np.nan
cm_vel_all.loc[ind_r6c_999,'alpha_m_err_r6c'] = np.nan
cm_vel_all.loc[ind_l6s_999,'alpha_m_err_l6s'] = np.nan

# ASPCAP flag? Remove them later when making plots. We don't use them for clustering anyway
# bad_bits = 2**23

In [4]:
# calculate the columns for the scaled action diamond
orb_param_all['Jtot'] = np.sqrt(orb_param_all['Jphi']**2+orb_param_all['JR']**2+orb_param_all['Jz']**2)
orb_param_all['diamond_x']=orb_param_all['Jphi']/orb_param_all['Jtot']
orb_param_all['diamond_y']=(orb_param_all['Jz']-orb_param_all['JR'])/orb_param_all['Jtot']

# Calculate the error for Jtot and diamond_x/y
orb_param_all['e_Jtot'] = np.sqrt(orb_param_all['Jphi']**2*orb_param_all['e_Jphi']**2+orb_param_all['JR']**2*orb_param_all['e_JR']**2+orb_param_all['Jz']**2*orb_param_all['e_Jz']**2)/orb_param_all['Jtot']
orb_param_all['e_diamond_x']=np.sqrt(orb_param_all['Jtot']**2*orb_param_all['e_Jphi']**2+orb_param_all['e_Jtot']**2*orb_param_all['Jphi']**2)/orb_param_all['Jtot']**2
orb_param_all['e_diamond_y']=np.sqrt(orb_param_all['Jtot']**2*(orb_param_all['e_JR']**2+orb_param_all['e_Jz']**2)+orb_param_all['e_Jtot']**2*(orb_param_all['Jz']-orb_param_all['JR'])**2)/orb_param_all['Jtot']**2

# Calculate L_perp for clustering
orb_param_all['Lperp'] = np.sqrt(orb_param_all['Lx']**2+orb_param_all['Ly']**2)
orb_param_all['e_Lperp'] = np.sqrt(orb_param_all['Lx']**2*orb_param_all['e_Lx']**2+orb_param_all['Ly']**2*orb_param_all['e_Ly']**2)/orb_param_all['Lperp']

# This mean metallicity below is only used in the case when we actually want to cluster in or with a prior cut on metallicity
# To avoid systematic differences between spectroscopic surveys; don't mix metallicities from different surveys
# cm_vel_all['m_h_mean'] = np.nanmean(cm_vel_all[['m_h_ap17','m_h_l6s','m_h_r6c','m_h_gl3']].values, axis=1).T
cm_vel_all['m_h_mean'], cm_vel_all['e_m_h_mean'] = cm_vel_all['m_h_ap17'], cm_vel_all['m_h_err_ap17'] 

orb_param_all['PCA_X'] = np.empty(len(orb_param_all))*np.nan
orb_param_all['PCA_Y'] = np.empty(len(orb_param_all))*np.nan

In [5]:
# Tweak with the axes that go into PCA

scaler = 'Robust' # 'Standard' or 'Robust' or None
denoise = None # 'PCA' or 'AE' or None
algorithm = 'HDBSCAN' # 'OPTICS' or 'DBSCAN' or 'HDBSCAN' or 'AGG_n' or 'AGG_l' or 'GMM'
# Define what axes go into PCA
action = True
diamond = False
metallicity = False
velocity = False
cylindrical = True
position = False
energy = False
eccentricity = False
Lz = False
Lperp = False

# Define what axes go into GMM with PCA results
# By default, include whatever was not in PCA
action_add = not action
metallicity_add = not metallicity
velocity_add = not velocity
position_add = not position
energy_add = not energy
eccentricity_add = not eccentricity
Lz_add = not Lz
Lperp_add = not Lperp

# Manual override additional axes
# action_add = False
position_add = False
metallicity_add = False
velocity_add = False
energy_add = False
eccentricity_add = False
Lz_add = False
Lperp_add = False

In [6]:
# My selection
cutoff = 2500
# Pick the stars with reasonable velocities(need to jutify this later)
# and reasoanable metallicity 
# and meet all quality cuts except for the binary cut
# Temporarily ignoring any action flags and not using any action for now
kin_qual = ((abs(cm_vel_all['U_g2']) < 1000) & 
            (abs(cm_vel_all['V_g2']) < 1000) & (abs(cm_vel_all['W_g2']) < 1000) & 
            (cm_vel_all['qual_flag'] == 0) # & 
           )
feh_qual = (((cm_vel_all['m_h_ap17'] > -10.0) | (cm_vel_all['m_h_r6c'] > -10.0) |
             (cm_vel_all['m_h_gl3'] > -10.0) | (cm_vel_all['m_h_l6s'] > -10.0)) &
            ((cm_vel_all['m_h_err_ap17'] > 0.0) | (cm_vel_all['m_h_err_r6c'] > 0.0) |
             (cm_vel_all['m_h_err_gl3'] > 0.0) | (cm_vel_all['m_h_err_l6s'] > 0.0)))

# act_qual = ((orb_param_all['flag_fail'] == 0) &
#             (orb_param_all['flag_unbound'] == 0) & 
#             (orb_param_all['flag_circ'] == 0) &
#             (orb_param_all['flag_act_conv'] == 0)
#            )


# selection = ((abs(orb_param_all['zmax']) > cutoff) & (cm_vel_all['vphi_g2'] > 100))
# selection = ((abs(orb_param_all['zmax']) > cutoff) & (cm_vel_all['m_h_mean'] < -1.))
selection = ((orb_param_all['zmax']-2*orb_param_all['e_zmax']) > cutoff)

if metallicity == False & metallicity_add == False:
    combined_cut = kin_qual & selection
else:
    print('Applying metallicity quality cut...')
    combined_cut = kin_qual & selection & feh_qual

ind_cut = np.where(combined_cut)[0]
print(len(ind_cut))

27921


In [7]:
# Cut the dataframe
df_cut_vel = cm_vel_all.loc[ind_cut,:]
df_cut_orb = orb_param_all.loc[ind_cut,:]

In [8]:
extratext = '0202_g2only_err'

if denoise == 'PCA':
    if action_add == True:
        extratext += '_act'

    if metallicity_add == True:
        extratext += '_feh'

    if velocity_add == True:
        extratext += '_vel'

    if position_add == True:
        extratext += '_pos'

    if energy_add == True:
        extratext += '_etot'

    if eccentricity_add == True:
        extratext += '_ecc'

    if Lz_add == True:
        extratext += '_Lz'
        
    if Lperp_add == True:
        extratext += '_Lperp'
else:
    if action == True or action_add == True:
        extratext += '_act'
        if diamond == True:
            extratext += '_diamond'

    if metallicity == True or metallicity_add == True:
        extratext += '_feh'

    if velocity == True or velocity_add == True:
        extratext += '_vel'
        if cylindrical == True:
            extratext += '_cyl'

    if position == True or position_add == True:
        extratext += '_pos'

    if energy == True or energy_add == True:
        extratext += '_etot'

    if eccentricity == True or eccentricity_add == True:
        extratext += '_ecc'

    if Lz == True or Lz_add == True:
        extratext += '_Lz'
        
    if Lperp == True or Lperp_add == True:
        extratext += '_Lperp'


extratext += '_'+str(cutoff)


# Make a list that records what axes are fed into PCA (or not)
pca_list = []


# Add in the dimensions as needed 
ydata_ini = []
ydata_ini_err = []

if action == True and diamond == True:
    ydata_ini.append(df_cut_orb['diamond_x'])
    ydata_ini.append(df_cut_orb['diamond_y'])
    ydata_ini_err.append(df_cut_orb['e_diamond_x'])
    ydata_ini_err.append(df_cut_orb['e_diamond_y'])
    pca_list.append('Act_diamond')
    
if action == True and diamond == False:
    ydata_ini.append(df_cut_orb['JR'])
    ydata_ini.append(df_cut_orb['Jphi'])
    ydata_ini.append(df_cut_orb['Jz'])
    ydata_ini_err.append(df_cut_orb['e_JR'])
    ydata_ini_err.append(df_cut_orb['e_Jphi'])
    ydata_ini_err.append(df_cut_orb['e_Jz'])
    pca_list.append('Act_3d')

    
if velocity == True and cylindrical == False:
    ydata_ini.append(df_cut_vel["U_g2"])
    ydata_ini.append(df_cut_vel["V_g2"])
    ydata_ini.append(df_cut_vel["W_g2"])
    ydata_ini_err.append(df_cut_vel["Uerr_g2"])
    ydata_ini_err.append(df_cut_vel["Verr_g2"])
    ydata_ini_err.append(df_cut_vel["Werr_g2"])
    pca_list.append('Vel_cart')
    
if metallicity == True:
    ydata_ini.append(df_cut_vel['m_h_mean'])
    ydata_ini_err.append(df_cut_vel['e_m_h_mean'])
    pca_list.append('[Fe/H]')
    
if velocity == True and cylindrical == True:
    ydata_ini.append(df_cut_vel["vr_g2"])
    ydata_ini.append(df_cut_vel["vphi_g2"])
    ydata_ini.append(df_cut_vel["vz_g2"])
    ydata_ini_err.append(df_cut_vel["vrerr_g2"])
    ydata_ini_err.append(df_cut_vel["vphierr_g2"])
    ydata_ini_err.append(df_cut_vel["vzerr_g2"])
    pca_list.append('Vel_cyl')
    
if position == True:
    ydata_ini.append(df_cut_vel["XGC"])
    ydata_ini.append(df_cut_vel["YGC"])
    ydata_ini.append(df_cut_vel["ZGC"])
    ydata_ini_err.append(df_cut_vel["XGCerr"])
    ydata_ini_err.append(df_cut_vel["YGCerr"])
    ydata_ini_err.append(df_cut_vel["ZGCerr"])
    pca_list.append('Pos_cart')

if energy == True:
    ydata_ini.append(df_cut_orb["Etot"])
    ydata_ini_err.append(df_cut_orb["e_Etot"])
    pca_list.append('E_tot')

if eccentricity == True:
    ydata_ini.append(df_cut_orb["ecc"])
    ydata_ini_err.append(df_cut_orb["e_ecc"])
    pca_list.append('ecc')

if Lz == True:
    ydata_ini.append(df_cut_orb["Lz"])
    ydata_ini_err.append(df_cut_orb["e_Lz"])
    pca_list.append('Lz')

if Lperp == True:
    ydata_ini.append(df_cut_orb["Lperp"])
    ydata_ini_err.append(df_cut_orb["e_Lperp"])
    pca_list.append('Lperp')
    


ydata = np.array(ydata_ini).T
ydata_err = np.array(ydata_ini_err).T

    

if scaler == 'Standard':
    X = StandardScaler().fit_transform(ydata)
elif scaler == 'Robust':
    X = RobustScaler().fit_transform(ydata)
elif scaler == None:
    X = ydata
  


print("Axes not in PCA:",pca_list)
# Put some place holder into the orbital param df
df_cut_orb['PCA_X'] = np.empty(len(df_cut_orb))*np.nan
df_cut_orb['PCA_Y'] = np.empty(len(df_cut_orb))*np.nan
df_cut_orb['e_PCA_X'] = np.empty(len(df_cut_orb))*np.nan
df_cut_orb['e_PCA_Y'] = np.empty(len(df_cut_orb))*np.nan
axes_labels = pca_list.copy()



# X_tp = X.T

# # Combine the result with the non-PCA axes and store the correponding uncertainties into a separate 
# # Do vstack with .T transpose twice!
# if action_add == True and diamond == True:
#     X_tp = np.vstack((X_tp,df_cut_orb['diamond_x']))
#     X_tp = np.vstack((X_tp,df_cut_orb['diamond_y']))
#     X_err_tp = np.vstack((X_err_tp,df_cut_orb['e_diamond_x']))
#     X_err_tp = np.vstack((X_err_tp,df_cut_orb['e_diamond_y']))
#     axes_labels.append('Act_diamond')
    
# if action_add == True and diamond == False:
#     X_tp = np.vstack((X_tp,df_cut_orb['JR']))
#     X_tp = np.vstack((X_tp,df_cut_orb['Jphi']))
#     X_tp = np.vstack((X_tp,df_cut_orb['Jz']))
#     X_err_tp = np.vstack((X_err_tp,df_cut_orb['e_JR']))
#     X_err_tp = np.vstack((X_err_tp,df_cut_orb['e_Jphi']))
#     X_err_tp = np.vstack((X_err_tp,df_cut_orb['e_Jz']))
#     axes_labels.append('Act_3d')

# if metallicity_add == True:
#     X_tp = np.vstack((X_tp,df_cut_vel['m_h_mean']))
#     X_err_tp = np.vstack((X_err_tp,df_cut_vel['e_m_h_mean']))
#     axes_labels.append('[Fe/H]')
    
# if velocity_add == True and cylindrical == False:
#     X_tp = np.vstack((X_tp,df_cut_vel["U_g2"]))
#     X_tp = np.vstack((X_tp,df_cut_vel["V_g2"]))
#     X_tp = np.vstack((X_tp,df_cut_vel["W_g2"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_vel["Uerr_g2"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_vel["Verr_g2"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_vel["Werr_g2"]))
#     axes_labels.append('Vel_cart')
    
# if velocity_add == True and cylindrical == True:
#     X_tp = np.vstack((X_tp,df_cut_vel["vr_g2"]))
#     X_tp = np.vstack((X_tp,df_cut_vel["vphi_g2"]))
#     X_tp = np.vstack((X_tp,df_cut_vel["vz_g2"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_vel["vrerr_g2"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_vel["vphierr_g2"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_vel["vzerr_g2"]))
#     axes_labels.append('Vel_cyl')
    
# if position_add == True:
#     X_tp = np.vstack((X_tp,df_cut_vel["XGC"]))
#     X_tp = np.vstack((X_tp,df_cut_vel["YGC"]))
#     X_tp = np.vstack((X_tp,df_cut_vel["ZGC"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_vel["XGCerr"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_vel["YGCerr"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_vel["ZGCerr"]))
#     axes_labels.append('Pos_cart')

# if energy_add == True:
#     X_tp = np.vstack((X_tp,df_cut_orb["Etot"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_orb["e_Etot"]))
#     axes_labels.append('E_tot')

# if eccentricity_add == True:
#     X_tp = np.vstack((X_tp,df_cut_orb["ecc"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_orb["e_ecc"]))
#     axes_labels.append('ecc')

# if Lz_add == True:
#     X_tp = np.vstack((X_tp,df_cut_orb["Lz"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_orb["e_Lz"]))
#     axes_labels.append('Lz')

# if Lperp_add == True:
#     X_tp = np.vstack((X_tp,df_cut_orb["Lperp"]))
#     X_err_tp = np.vstack((X_err_tp,df_cut_orb["e_Lperp"]))
#     axes_labels.append('Lperp')
    
    
# Put everything through Scaler again
# if scaler == 'Standard':
#     X = StandardScaler().fit_transform(X_tp.T)
#     X_err = StandardScaler().fit(X_tp.T).transform(X_err_tp.T)
# elif scaler == 'Robust':
#     X = RobustScaler().fit_transform(X_tp.T)
#     X_err = RobustScaler().fit(X_tp.T).transform(X_err_tp.T)
# elif scaler == None:
#     X = X_tp.T
#     X_err = X_err_tp.T
    
print("Final clustering axes:",axes_labels)
print("Shape of X after adding additional axes:",np.shape(X))
print("Extra text is:",extratext)

Axes not in PCA: ['Act_3d']
Final clustering axes: ['Act_3d']
Shape of X after adding additional axes: (27921, 3)
Extra text is: 0202_g2only_err_act_2500


In [9]:
if denoise == 'PCA':
    print('counting PCA...')
    n_dim=2
else:
    n_dim=0
    if metallicity == True:
        print('counting Metallicity...')
        n_dim += 1
    if action == True and diamond == True:
        print('counting action diamond...')
        n_dim += 2
    if action == True and diamond == False:
        print('counting 3d action...')
        n_dim += 3
    if velocity == True and cylindrical == False:
        print('counting cartesian velocity...')
        n_dim += 3
    if velocity == True and cylindrical == True:
        print('counting cylindrical velocity...')
        n_dim += 3
    if position == True:
        print('counting positions...')
        n_dim += 3
    if energy == True:
        print('counting energy...')
        n_dim += 1
    if eccentricity == True:
        print('counting eccentricity...')
        n_dim += 1
    if Lz == True:
        print('counting Lz...')
        n_dim += 1
    if Lperp == True:
        print('counting Lperp...')
        n_dim += 1

if metallicity_add == True:
    print('counting Metallicity...')
    n_dim += 1
if action_add == True and diamond == True:
    print('counting action diamond...')
    n_dim += 2
if action_add == True and diamond == False:
    print('counting 3d action...')
    n_dim += 3
if velocity_add == True:
    print('counting cartesian velocity...')
    n_dim += 3
if position_add == True:
    print('counting positions...')
    n_dim += 3
if energy_add == True:
    print('counting energy...')
    n_dim += 1
if eccentricity_add == True:
    print('counting eccentricity...')
    n_dim += 1
if Lz_add == True:
    print('counting Lz...')
    n_dim += 1
if Lperp_add == True:
    print('counting Lperp...')
    n_dim += 1
    
print(n_dim)

counting 3d action...
3


In [10]:
# Clustering options
extratext += '_hdbscan'
cluster_selection_method = 'eom'
extratext += '_'+str(cluster_selection_method)

In [11]:
plots_dir = './tree_plot_act_3d_2500/eom/'

In [12]:
min_samples_arr = np.arange(2,20,2)
min_cluster_size_arr = np.arange(20,200,20)

In [13]:
print(min_samples_arr,min_cluster_size_arr)
print(extratext)

[ 2  4  6  8 10 12 14 16 18] [ 20  40  60  80 100 120 140 160 180]
0202_g2only_err_act_2500_hdbscan_eom


In [14]:
for i in range(len(min_samples_arr)):
    for j in range(len(min_cluster_size_arr)):
        # Apply HDBSCAN
        min_samples = int(min_samples_arr[i])
        min_cluster_size = int(min_cluster_size_arr[j])
        extratext_f = extratext + '_min_samples_'+str(min_samples)
        extratext_f += '_min_cluster_size_'+str(min_cluster_size)
        clust = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,  
                        min_samples=min_samples,
                        gen_min_span_tree=True, 
                        cluster_selection_method=cluster_selection_method).fit(X)
        labels = clust.labels_ #[clust.ordering_]


        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)

        print('For min_samples=',min_samples,' and min_cluster_size=',min_cluster_size)
        print('Estimated number of clusters: %d' % n_clusters_)
        print('Estimated number of noise points: %d' % n_noise_)


        # ###########################################################################

        # Black removed and is used for noise instead.
        # unique_labels = set(labels)
        # print("unique_labels are", unique_labels)


        # for k in unique_labels:
        #     print("k is", k)

        #     class_member_mask = (labels == k)

        #     if algorithm == 'DBSCAN':
        #         mask = class_member_mask & core_samples_mask
        #         anti_mask = class_member_mask & ~core_samples_mask
        #     elif algorithm == 'OPTICS' or algorithm == 'HDBSCAN' or algorithm == 'AGG_n' or algorithm == 'AGG_l' or algorithm == 'GMM':
        #         mask = class_member_mask
        #         anti_mask = class_member_mask

        #     xy = X[mask]
        #     cluster_means[k] = np.mean(xy, axis = 0)
        #     cluster_dispersions[k] = np.std(xy, axis = 0)
        #     cluster_nstars[k] = len(X[class_member_mask])


        # Generate the hierarchical tree if HDBSCAN
        f = plt.figure(figsize=[20,10])
        clust.condensed_tree_.plot(select_clusters=True)
        f.savefig(plots_dir + 'hdbscan_clustering' + extratext_f + '_tree' + '.pdf')
        plt.close(f)



For min_samples= 2  and min_cluster_size= 20
Estimated number of clusters: 6
Estimated number of noise points: 1262
For min_samples= 2  and min_cluster_size= 40
Estimated number of clusters: 3
Estimated number of noise points: 970
For min_samples= 2  and min_cluster_size= 60
Estimated number of clusters: 2
Estimated number of noise points: 2075
For min_samples= 2  and min_cluster_size= 80
Estimated number of clusters: 8
Estimated number of noise points: 11448
For min_samples= 2  and min_cluster_size= 100
Estimated number of clusters: 4
Estimated number of noise points: 11791
For min_samples= 2  and min_cluster_size= 120
Estimated number of clusters: 2
Estimated number of noise points: 12016
For min_samples= 2  and min_cluster_size= 140
Estimated number of clusters: 2
Estimated number of noise points: 12016
For min_samples= 2  and min_cluster_size= 160
Estimated number of clusters: 2
Estimated number of noise points: 12016
For min_samples= 2  and min_cluster_size= 180
Estimated number o